In [ ]:
from ultralytics import YOLO
import cv2
from torch_snippets import *

model = YOLO("yolo11n.pt")

cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break

    res = model(frame)[0].plot()
    cv2.imshow("test", res)

    k = cv2.waitKey(1)
    if k%256 == 27:
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()
cv2.destroyAllWindows()

<h1>Remember How to deploy on Heruko later! Time last updated: 3:06:06 AM</h1>

In [ ]:
import cv2 #type:ignore
from torch_snippets import * #type:ignore
from skimage.transform import resize #type:ignore
from ultralytics import YOLO #type:ignore
import numpy as np #type:ignore
import datetime
import random

model = YOLO("yolo11n-seg.pt") 

cam = cv2.VideoCapture(0) 

alpha = 0.5 

class_colors = {
    "person": [0, 0, 255], 
    'cell phone': [0, 255, 0], 
    "chair": [255, 0, 0],  
}

def get_class_color(class_name):
    return class_colors.get(class_name, [random.randint(0, 255) for _ in range(3)])

while True:
    ret, frame = cam.read()  
    if not ret:
        print("Cam not found")
        break

    frame_resized = cv2.resize(frame, (1024, 768))  

    res = model(frame_resized)[0]  

    if res.masks is not None and len(res.masks) > 0:
        highlighted_frame = frame.copy() 

        for i in range(len(res.masks)):
            mask = res.masks.data[i].cpu().numpy() 
            mask_binary = (mask > 0).astype(int)  

            mask_resized = resize(mask_binary, (frame.shape[0], frame.shape[1]), mode='reflect', anti_aliasing=True)

            class_idx = int(res.boxes[i].cls)
            class_name = res.names[class_idx]  

            
            color = get_class_color(class_name)  
            mask_colored = np.zeros_like(frame)  
            mask_colored[mask_resized > 0] = color  

            highlighted_frame[mask_resized > 0] = cv2.addWeighted(frame, 1 - alpha, mask_colored, alpha, 0)[mask_resized > 0]

        cv2.imshow('Camera', highlighted_frame)
    else:
        cv2.imshow('Camera', frame)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:
        break

    if key == ord(" "): 
        filename = f'{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.png'
        cv2.imwrite(f'{filename}', frame)
        cv2.imshow('Saved', frame)
        show(read(f'{filename}'))
        print(f"Image saved as {filename}") 

cam.release()
cv2.destroyAllWindows()